# 
# Utilização de Algorítmos de Machine Learning para Identificação de Empresas "de Fachada" em Operações de Importação
TCC PUC Minas - LUCIANA MACEDO RODRIGUES

### NOTEBOOK 2 - TRATAMENTO DOS DADOS

### 1 - Configurações iniciais

In [ ]:
# Importando os pacotes necessários
import pandas as pd
import numpy as np
import datetime
from datetime import date

### 2 - Leitura das bases de dados

In [ ]:
df_merged = pd.read_csv('arquivos/df_merged.csv', sep = ',')

In [ ]:
df_merged.head()

### 3 - Tratamento dos dados

### 3.1 - Valores nulos

In [ ]:
df_merged.isnull().sum()

Os 307 valores nulos na coluna 'EMP_REC' decorrem da ausência de Receita Brusta Estimada para essas empresas (conforme evidenciado no NOTEBOOK01 pela diferença do número de linhas de dadosADU = 20.953 e dadosRB = 20.646), portanto a melhor forma de representá-los é com o numeral 0, ou seja: nenhuma Receita Bruta Estimada no período.

O mesmo ocorre com os 1.937 valores nulos na coluna 'EMP_EMPREG' que decorrem da ausência de dados de empregados em dadosDIRF para essas empresas (conforme evidenciado no NOTEBOOK01 pela diferença do número de linhas de dadosADU = 20.953 e dadosDIRF = 19.016), que também serão substistituidos pelo o numeral 0, indicando nenhum empregado no período.

In [ ]:
# Substituindo os valores NaN por 0 na coluna 'EMP_RB'
df_merged['EMP_REC'].fillna(0, inplace = True)
df_merged['EMP_REC'].isnull().sum()

In [ ]:
# Substituindo os valores NaN por 0 na coluna 'EMP_EMPREG'
df_merged['EMP_EMPREG'].fillna(0, inplace = True)
df_merged['EMP_EMPREG'].isnull().sum()

O valor nulo na coluna "DT_ABERT_EMP" não é esperado, por isso verificamos essa linha e por se tratar de uma empresa ativa, optamos por descartá-la, sem comprometer o modelo. Caso fosse uma das poucas instâncias das empresas "alvo", poderíamos estudar outro tratamento. 

In [ ]:
# Localizando o índice das linhas de df_merged que contém valor nulo na coluna DT_ABERT_EMP
np.where(df_merged["DT_ABERT_EMP"].isnull())

In [ ]:
# Verificando os demais dados dessa linha
linha = df_merged.iloc[[12064]]
linha

In [ ]:
# Eliminando a linha de index 12064
df_merged.drop(index=[12064],inplace=True)
df_merged['DT_ABERT_EMP'].isnull().sum()

Os 555 valores nulos na coluna "DT_SIT_CAD_EMP" também não são esperados, representando algum erro no dataset original. 
Optamos por descartar essas linhas.

In [ ]:
# Eliminando as 555 linhas que contém dados faltantes na coluna "DT_SIT_CAD_EMP". O df_merged passará a ter só 20.397 linhas.
df_merged.dropna(subset=['DT_SIT_CAD_EMP'], inplace=True)
df_merged['DT_SIT_CAD_EMP'].isnull().sum()

In [ ]:
df_merged.isnull().sum()

### 3.2 - Tipos dos dados

In [ ]:
# Tipos de Dados
df_merged.dtypes

In [ ]:
# Corrigindo os dtype das colunas que se referem à datas no df_merged
df_merged = df_merged.astype({'DT_ABERT_EMP': 'datetime64',
                              'DT_SIT_CAD_EMP' : 'datetime64'})

In [ ]:
df_merged.head()

In [ ]:
# Arrumar os dtypes de 'EMP_REC' e 'EMP_EMPREG' para int64
df_merged = df_merged.astype({'EMP_REC': 'int64'})

Ao tentar converter os tipos de dados da coluna "EMP_REC", verificamos que havia várias linhas com o valor nulo preenchidas com ' -   ' interpretadas como objetos pelo pandas.

In [ ]:
# Verificando as linhas com problema
np.where(df_merged['EMP_REC'] == ' -   ')

In [ ]:
# Convertendo os dados da coluna EMP_REC' para números e NaN
df_merged['EMP_REC']=pd.to_numeric(df_merged['EMP_REC'], errors='coerce')

In [ ]:
# substituindo os NaN por "0"
df_merged['EMP_REC'].fillna(0, inplace = True)

In [ ]:
# Corrigindo o tipo de dados em 'EMP_REC' para int64
df_merged = df_merged.astype({'EMP_REC': 'int64'})

In [ ]:
# Corrigindo o tipo de dados em 'EMP_EMPREG' para int64
df_merged = df_merged.astype({'EMP_EMPREG': 'int64'})

In [ ]:
# Verificando os tipos de dados após alterações
df_merged.dtypes

In [ ]:
# Verificando a integridade dos dados
df_merged.head()

### 3.3 Classes das variáveis categóricas

In [ ]:
# Verificando as classes da variável categórica 'UF_EMP'
freq_df_merged = df_merged.groupby(['UF_EMP']).size()
freq_df_merged

In [ ]:
# Verificando as classes da variável categórica 'EMP_PORTE'
freq_df_merged = df_merged.groupby(['EMP_PORTE']).size()
freq_df_merged

A classe 'inválido' representa um erro do dataset original situado dentro da classe das empresas alvo, como evidenciado abaixo. Assim, optamos por substituí-lo pelo valor da classe dominante, ao invés de excluir a linha.

In [ ]:
np.where(df_merged['EMP_PORTE'] == 'INVALIDO')

In [ ]:
# Verificando os demais dados dessa linha
linha = df_merged.iloc[[13707]]
linha

In [ ]:
# Substituindo o valor INVÁLIDO pela classe dominante da variável categórica
df_merged = df_merged.replace('INVALIDO', 'DEMAIS')

### 3.4 - Definição e formatação da variável TARGET.

A variável target do estudo consiste na combinação dos valores da coluna SIT_CAD_EMP (que representam a situação cadastral da empresa),  com os valores da coluna MOT_SIT_CAD_EMP (que representam o motivo daquela situação cadastral).

Consideraremos "empresas de fachada", portanto "positivas", aquelas com situação cadastral diferente de "ativa", desde que os motivos atribuidos àquelas situações sejam decorrentes de irregularidades fiscais, representados por: "inexistente de fato", 
"localização desconhecida", "omissão de declarações", "prática irregular de operação de comércio exterior","inconsistência cadastral".

Para tanto, atribuiremos valores "1" para as variáveis de interesse acima e "0" para as demais em ambas as colunas, substituindo os valores orginais e criando uma nova coluna "TARGET" fruto da multiplicação das duas colunas de forma que as variáveis resultantes sejam "1" para casos positivos e "0" para os casos negativos.

In [ ]:
# Substituindo os dados 
dict_subst = {"ATIVA": 0, "BAIXADA": 1, "INAPTA": 1, "SUSPENSA": 1, "SEM MOTIVO": 0, 
              "EXTINCAO POR ENCERRAMENTO LIQUIDACAO VOLUNTARIA": 0, "INCORPORACAO": 0, 
              "INEXISTENTE DE FATO": 1, "LOCALIZACAO DESCONHECIDA": 1, "OMISSAO DE DECLARACOES": 1,
              "PRATICA IRREGULAR DE OPERACAO DE COMERCIO EXTERIOR": 1, "INCONSISTENCIA CADASTRAL": 1,
              "INTERRUPCAO TEMPORARIA DAS ATIVIDADES": 0, "PEDIDO DE BAIXA INDEFERIDA": 0}
df_merged = df_merged.replace(dict_subst)

In [ ]:
# Verificando alterações
freq_df_merged = df_merged.groupby(['SIT_CAD_EMP', 'MOT_SIT_CAD_EMP']).size()
freq_df_merged

In [ ]:
# Obtendo todas as informações de df_merged. O tipo de dados das duas últimas colunas agora é int64
df_merged.info()

In [ ]:
# Criando a coluna TARGET
df_merged['TARGET'] = df_merged['SIT_CAD_EMP'] * df_merged['MOT_SIT_CAD_EMP']
df_merged.head()

In [ ]:
# Verificando alterações
freq_df_merged = df_merged.groupby(['SIT_CAD_EMP', 'MOT_SIT_CAD_EMP', 'TARGET']).size()
freq_df_merged

### 3.5 - Definição e formatação da variável ANOS_ATIVA

Desejamos trazer ao estudo o tempo de atividade das empresas analisadas, expresso em anos, o que faremos usando o módulo datetime.

Para as empresas ativas, faremos o cálculo subtraindo da data atual os valores da coluna "DT_ABERT_EMP", para as demais faremos o cálculo subtraindo dos dados da coluna "DT_SIT_CAD_EMP" que representa a data da última alteração cadastral das empresa e, portanto, equivale à data de encerramento/suspensão das suas atividades.

In [ ]:
# Criando uma coluna 'DATA_ATUAL'
data_atual = date.today()
df_merged['DATA_ATUAL'] = data_atual
df_merged = df_merged.astype({'DATA_ATUAL': 'datetime64'})
df_merged.head()

In [ ]:
df_merged.info()

In [ ]:
# Criando a coluna 'ANOS_ATIVA' com a primeira condição
df_merged.loc[df_merged['SIT_CAD_EMP'] != 0, 'ANOS_ATIVA' ] = ((df_merged['DT_SIT_CAD_EMP'] - df_merged['DT_ABERT_EMP'])
                                                            /np.timedelta64(1 , 'Y')).astype(int)
df_merged.head()

In [ ]:
# Preenchendo a coluna 'ANOS_ATIVA' com a segunda condição
df_merged.loc[df_merged['SIT_CAD_EMP'] == 0, 'ANOS_ATIVA' ] = ((df_merged['DATA_ATUAL'] - df_merged['DT_ABERT_EMP'])
                                                            /np.timedelta64(1 , 'Y')).astype(int)
df_merged.head()

In [ ]:
df_merged = df_merged.astype({'ANOS_ATIVA': 'int64'})

### 3.6 - Organizando e salvando o df_final 

Eliminaremos as colunas desnecessárias criando o dataframe final que será utilizado no modelo.

In [ ]:
# Criando df_final
df_final = df_merged.copy()

In [ ]:
# "Limpando" df_final (RASCUNHO)
df_final.drop(columns=['DT_ABERT_EMP', 'DT_SIT_CAD_EMP', 'SIT_CAD_EMP', 'MOT_SIT_CAD_EMP','DATA_ATUAL'],inplace=True)

In [ ]:
# Organizando as colunas no df_final
df_final = df_final[['UF_EMP', 'EMP_PORTE', 'EMP_CS_MAX', 'IMP_VOL',
                       'IMP_VAL', 'IMP_PESO', 'IMP_CUSTO', 'EMP_EMPREG', 'EMP_REC', 'ANOS_ATIVA', 
                       'TARGET']]

In [ ]:
# Criando csv de df_final
df_final.to_csv("arquivos/df_final.csv", encoding = 'utf-8', index = False)

In [ ]:
# Verificando csv de df_final
df_final = pd.read_csv('arquivos/df_final.csv', sep = ',')
df_final.head()